# Hartree-Fock VQE Demo

In [ ]:
import tqdm
import pickle

import numpy as np
import scipy as sp
import numpy_ml

from molecular_data import MolecularData, obi_basis_rotation, tbi_basis_rotation, generate_hamiltonian
from QuICT.algorithm.quantum_machine_learning.model.VQE.operators.encoder import JordanWigner
from QuICT.algorithm.quantum_machine_learning.model.VQE import HartreeFockVQENet
from QuICT.algorithm.quantum_machine_learning.utils import Hamiltonian

## Load the data

In [ ]:
moldir = "./molecular_data"
molfile = moldir + "/H6_sto-3g_singlet_linear_r-1.3.hdf5"
moldata = MolecularData(molfile)

overlap = np.load(moldir + "/overlap.npy")
Hcore = np.load(moldir + "/h_core.npy")
two_electron_integral = np.einsum("psqr", np.load(moldir + "/tei.npy"))  # (1, 1, 0, 0)

_, X = sp.linalg.eigh(Hcore, overlap)
obi = obi_basis_rotation(Hcore, X)
tbi = tbi_basis_rotation(two_electron_integral, X)
molecular_hamiltonian = generate_hamiltonian(moldata.nuclear_repulsion, obi, tbi)

## Convert the Hamiltonian

In [ ]:
fermi_op = molecular_hamiltonian.get_fermion_operator()
orbitals = 2 * moldata.n_orbitals
electrons = moldata.n_electrons
qubit_op = JordanWigner(orbitals).encode(fermi_op)
hamiltonian = Hamiltonian(qubit_op.to_hamiltonian())

ham_str = pickle.dumps(hamiltonian)
with open('hamiltonian', 'wb') as f:
    f.write(ham_str)

## Calculate the ground energy

In [ ]:
MAX_ITERS = 1000
LR = 0.1

orbitals = 2 * moldata.n_orbitals
electrons = moldata.n_electrons
with open('hamiltonian', 'rb') as f:
    hamiltonian = pickle.load(f)
hfvqe_net = HartreeFockVQENet(orbitals, electrons, hamiltonian)
optim = numpy_ml.neural_nets.optimizers.Adam(lr=LR)
energy = []

loader = tqdm.trange(MAX_ITERS, desc="Training", leave=True)
for it in loader:
    state, loss = hfvqe_net.run_step(optim)
    print(hfvqe_net.params)
    for g in hfvqe_net._circuit.gates:
        print(g.type, g.parg)
    loader.set_postfix(loss=loss.item())

print(energy)
print(hfvqe_net.params)
print(float(loss))